## 範例 : (Kaggle)房價預測精簡版
- https://www.kaggle.com/c/house-prices-advanced-regression-techniques

- 以下是房價預測的精簡版範例，使用最小量的特徵工程以及線性回歸模型做預測, 最後輸出可以在Kaggle提交的預測檔


In [5]:
#載入套件
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

#讀取訓練與測試資料
data_path = '/home/edith/文件/100days-data/'
df_test = pd.read_csv(data_path + 'house_test.csv')
df_train = pd.read_csv(data_path + 'house_train.csv')
df_train.head()
# print(df_train.shape)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
#訓練資料需要train_X, train_Y / 預測輸出需要ids(識別每一個預測值), test_X
#在此先抽離出 train_Y與ids, 而先將train_X, test_X該有的資料合併成df, 先做特徵工程
train_Y = np.log1p(df_train['SalePrice'])
ids = df_test['Id']
df_train = df_train.drop(['Id', 'SalePrice'], axis=1) #axis=1表示要丟掉Id與SalePrice這兩"行"
# df_train.head()
df_test = df_test.drop(['Id'], axis=1)
df = pd.concat([df_train, df_test])#相同字段的表首尾相接(表示合併後數據變長，如果無此項目，會以ΝaN表示)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [7]:
df.isnull().sum()# 檢查每一個項目有幾個NaN

MSSubClass          0
MSZoning            4
LotFrontage       486
LotArea             0
Street              0
Alley            2721
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
MasVnrType         24
MasVnrArea         23
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           81
                 ... 
HalfBath            0
BedroomAbvGr        0
KitchenAbvGr        0
KitchenQual         1
TotRmsAbvGrd        0
Functional          2
Fireplaces          0
FireplaceQu      1420
GarageType        157
GarageYrBlt       159
GarageFinish      159
GarageCars          1
GarageArea          1
GarageQual        159
GarageCond

In [8]:
#特徵工程-簡化版：全部空值先填補-1，所有類別欄位先做LabelEncoder，然後再與數字欄位做MinMaxScaler
#這區塊的細節會在後續的課程做解說
LEncoder = LabelEncoder()
MMEncoder = MinMaxScaler()
for c in df.columns:
    df[c] = df[c].fillna(-1)
    if df[c].dtype == 'object':
        df[c] = LEncoder.fit_transform(list(df[c].values))
    df[c] = MMEncoder.fit_transform(df[c].values.reshape(-1,1))
df.head()

/home/edith/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.235294,0.8,0.210191,0.033420,1.0,0.0,1.0,1.0,0.5,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.50,1.0,0.8
1,0.000000,0.8,0.257962,0.038795,1.0,0.0,1.0,1.0,0.5,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.363636,0.25,1.0,0.8
2,0.235294,0.8,0.219745,0.046507,1.0,0.0,0.0,1.0,0.5,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.727273,0.50,1.0,0.8
3,0.294118,0.8,0.194268,0.038561,1.0,0.0,0.0,1.0,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.00,1.0,0.0
4,0.235294,0.8,0.270701,0.060576,1.0,0.0,0.0,1.0,0.5,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.50,1.0,0.8


In [9]:
#將前述轉換完畢資料df，重新切成train_X, test_X
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

In [10]:
train_X

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.235294,0.8,0.210191,0.033420,1.0,0.0,1.000000,1.000000,0.5,1.00,...,0.000000,0.0,0.0,0.00,0.00,0.000000,0.090909,0.50,1.000000,0.8
1,0.000000,0.8,0.257962,0.038795,1.0,0.0,1.000000,1.000000,0.5,0.50,...,0.000000,0.0,0.0,0.00,0.00,0.000000,0.363636,0.25,1.000000,0.8
2,0.235294,0.8,0.219745,0.046507,1.0,0.0,0.000000,1.000000,0.5,1.00,...,0.000000,0.0,0.0,0.00,0.00,0.000000,0.727273,0.50,1.000000,0.8
3,0.294118,0.8,0.194268,0.038561,1.0,0.0,0.000000,1.000000,0.5,0.00,...,0.000000,0.0,0.0,0.00,0.00,0.000000,0.090909,0.00,1.000000,0.0
4,0.235294,0.8,0.270701,0.060576,1.0,0.0,0.000000,1.000000,0.5,0.50,...,0.000000,0.0,0.0,0.00,0.00,0.000000,1.000000,0.50,1.000000,0.8
5,0.176471,0.8,0.273885,0.059899,1.0,0.0,0.000000,1.000000,0.5,1.00,...,0.000000,0.0,0.0,0.75,0.75,0.041176,0.818182,0.75,1.000000,0.8
6,0.000000,0.8,0.242038,0.041057,1.0,0.0,1.000000,1.000000,0.5,1.00,...,0.000000,0.0,0.0,0.00,0.00,0.000000,0.636364,0.25,1.000000,0.8
7,0.235294,0.8,0.000000,0.042450,1.0,0.0,0.000000,1.000000,0.5,0.00,...,0.000000,0.0,0.0,0.00,0.75,0.020588,0.909091,0.75,1.000000,0.8
8,0.176471,1.0,0.165605,0.022529,1.0,0.0,1.000000,1.000000,0.5,1.00,...,0.000000,0.0,0.0,0.00,0.00,0.000000,0.272727,0.50,1.000000,0.0
9,1.000000,0.8,0.162420,0.028605,1.0,0.0,1.000000,1.000000,0.5,0.00,...,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.50,1.000000,0.8


In [11]:
test_X

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.000000,0.6,0.257962,0.048246,1.0,0.0,1.000000,1.000000,0.5,1.00,...,0.208333,0.0,0.0,0.75,0.00,0.000000,0.454545,1.0,1.000000,0.8
1,0.000000,0.8,0.261146,0.060609,1.0,0.0,0.000000,1.000000,0.5,0.00,...,0.000000,0.0,0.0,0.00,0.25,0.735294,0.454545,1.0,1.000000,0.8
2,0.235294,0.8,0.238854,0.058566,1.0,0.0,0.000000,1.000000,0.5,1.00,...,0.000000,0.0,0.0,0.75,0.00,0.000000,0.181818,1.0,1.000000,0.8
3,0.235294,0.8,0.251592,0.040562,1.0,0.0,0.000000,1.000000,0.5,1.00,...,0.000000,0.0,0.0,0.00,0.00,0.000000,0.454545,1.0,1.000000,0.8
4,0.588235,0.8,0.140127,0.017318,1.0,0.0,0.000000,0.333333,0.5,1.00,...,0.250000,0.0,0.0,0.00,0.00,0.000000,0.000000,1.0,1.000000,0.8
5,0.235294,0.8,0.242038,0.040665,1.0,0.0,0.000000,1.000000,0.5,0.00,...,0.000000,0.0,0.0,0.00,0.00,0.000000,0.272727,1.0,1.000000,0.8
6,0.000000,0.8,0.000000,0.031223,1.0,0.0,0.000000,1.000000,0.5,1.00,...,0.000000,0.0,0.0,0.25,0.75,0.029412,0.181818,1.0,1.000000,0.8
7,0.235294,0.8,0.203822,0.033195,1.0,0.0,0.000000,1.000000,0.5,1.00,...,0.000000,0.0,0.0,0.00,0.00,0.000000,0.363636,1.0,1.000000,0.8
8,0.000000,0.8,0.273885,0.041487,1.0,0.0,1.000000,1.000000,0.5,1.00,...,0.000000,0.0,0.0,0.00,0.00,0.000000,0.090909,1.0,1.000000,0.8
9,0.000000,0.8,0.226115,0.033186,1.0,0.0,1.000000,1.000000,0.5,0.00,...,0.000000,0.0,0.0,0.75,0.00,0.000000,0.272727,1.0,1.000000,0.8


In [12]:
#使用線性回歸模型
#train_X, train_Y訓練模型， 並對test_X做出預測結果pred
from sklearn.linear_model import LinearRegression
estimator = LinearRegression()
estimator.fit(train_X, train_Y)
pred = estimator.predict(test_X)
pred

array([11.67036723, 11.97477756, 12.02198303, ..., 11.97917782,
       11.66362353, 12.38545966])

In [13]:
#將輸出結果pred與前面留下的ID(ids)合併，輸出成檔案
#可以下載並點開 house_baseline.csv 查看結果, 以便了解預測結果的輸出格式
#本範例所與作業所輸出的 csv 檔, 均可用於本題的 Kaggle 答案上傳, 可以試著上傳來熟悉 Kaggle 的介面操作
pred = np.expm1(pred)
sub = pd.DataFrame({'Id':ids, 'SalePrice':pred})
sub.to_csv('house_baseline.csv', index = True)#如果index＝False會沒有序號行

# 作業
- 下列A~E五個程式區塊中，哪一塊是特徵工程?
- 對照程式區塊 B 與 C 的結果，請問那些欄位屬於"類別型欄位"? (回答欄位英文名稱即可) 
- 續上題，請問哪個欄位是"目標值"?

In [22]:
# 程式區塊 A
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

#讀取訓練與測試資料
data_path = '/home/edith/文件/100days-data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.head()
# df_train.shape

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [23]:
# 程式區塊 B
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
# 程式區塊 C
LEncoder = LabelEncoder()
MMEncoder = MinMaxScaler()
for c in df.columns:
    df[c] = df[c].fillna(-1)
    if df[c].dtype == 'object':
        df[c] = LEncoder.fit_transform(list(df[c].values))
    df[c] = MMEncoder.fit_transform(df[c].values.reshape(-1, 1))
df.head()

/home/edith/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,0.118683,1.0,0.283951,0.125,0.0,0.775862,0.016072,0.000000,1.000000
1,0.0,0.218989,0.0,0.481481,0.125,0.0,0.879310,0.140813,0.575269,0.333333
2,1.0,0.400459,0.0,0.333333,0.000,0.0,0.984914,0.017387,0.000000,1.000000
3,0.0,0.323124,0.0,0.444444,0.125,0.0,0.070043,0.105390,0.381720,1.000000
4,1.0,0.016845,1.0,0.444444,0.000,0.0,0.699353,0.017630,0.000000,1.000000


In [25]:
# 程式區塊 D
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()
estimator.fit(train_X, train_Y)
pred = estimator.predict(test_X)

In [26]:
# 程式區塊 E
sub = pd.DataFrame({'PassengerId': ids, 'Survived': pred})
sub.to_csv('titanic_baseline.csv', index=False) 

#### 答案如下：
1. c
2. Pclass    Sex Ticket    Cabin    Embarked
3. Survived